DS 266 Final Project for BART model
The Colab Notebook link is https://colab.research.google.com/drive/1sckwqe2xo6B7m6deFPyA2cEKgscaQWeM?authuser=9#scrollTo=p3uZVUKssMhI

# Set up
Install and import libraries/dependencies

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install HuggingFace transformers
%pip install transformers

In [ ]:
%pip install evaluate

In [ ]:
%pip install rouge_score

In [ ]:
%pip install prometheus-eval

In [ ]:
%pip install triton

In [ ]:
%pip install vllm

In [ ]:
# Import BART
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import set_seed
from transformers import Trainer
from transformers import TrainingArguments
import random
from sklearn.model_selection import train_test_split
import evaluate
import re
from datasets import Dataset
from torch.utils.data import DataLoader, TensorDataset

# LLM as Eval Judge
from prometheus_eval.vllm import VLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE

## Set Random Seed
One may comment these out for random behaviors

In [ ]:
# Set seeds for reproducibility
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)
set_seed(seed_value)

# Datasets

In [ ]:
# Datasets file paths, may change per envrionments like local vs Google Colab
ROCStoriesSpring2016_FILE_PATH = './drive/MyDrive/DS 266/Final/ROCStoriesSpring2016.csv'
ROCStoriesWinter2017_FILE_PATH = './drive/MyDrive/DS 266/Final/ROCStoriesWinter2017.csv'
ClozeTestWinter2018ValData_FILE_PATH = './drive/MyDrive/DS 266/Final/ClozeTestWinter2018ValData.csv'
ClozeTestWinter2018TestData_FILE_PATH = './drive/MyDrive/DS 266/Final/ClozeTestWinter2018TestData.csv'
ClozeTestSpring2016ValData_FILE_PATH = './drive/MyDrive/DS 266/Final/ClozeTestSpring2016ValData.csv'

In [ ]:
# Load data
ROCStoriesSpring2016 = pd.read_csv(ROCStoriesSpring2016_FILE_PATH)
ROCStoriesWinter2017 = pd.read_csv(ROCStoriesWinter2017_FILE_PATH)
ClozeTestWinter2018ValData = pd.read_csv(ClozeTestWinter2018ValData_FILE_PATH)
ClozeTestWinter2018TestData = pd.read_csv(ClozeTestWinter2018TestData_FILE_PATH)
ClozeTestSpring2016ValData = pd.read_csv(ClozeTestSpring2016ValData_FILE_PATH)

## Inspect Datasets
Check shapes and duplicates

In [ ]:
print(ROCStoriesSpring2016.shape)
ROCStoriesSpring2016.head()

(45496, 7)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.


In [ ]:
print(ROCStoriesWinter2017.shape)
ROCStoriesWinter2017.head()

(52665, 7)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [ ]:
print(ClozeTestWinter2018ValData.shape)
ClozeTestWinter2018ValData.head()

(1571, 8)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1
1,bff9f820-9605-4875-b9af-fe6f14d04256,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.,1
2,e8f628d5-9f97-40ed-8611-fc0e774673c4,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...,2
3,f5226bfe-9f26-4377-b05f-3d9568dbdec1,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.,1
4,69ac9b05-b956-402f-9fff-1f926ef9176b,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.,1


In [ ]:
print(ClozeTestWinter2018TestData.shape)
ClozeTestWinter2018TestData.head()

(1571, 7)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,f6aad64a-e34c-415d-b895-dbfa187ed43e,Bob was bored at his job as a school teacher.,He had been working so hard this past month.,He decided to treat himself with something spe...,He ordered tickets for a weekend snowboarding ...,He was looking forward to getting away.,His boss told him he had to work this weekend.
1,0fedd90d-5295-4b79-b2d0-15a2bad624ee,Olivia went out with Harry on a date.,Harry thought the date was going well.,Olivia thinks he is a complete jerk and never ...,Harry keeps raving about their chemistry.,Olivia is about to leave.,"Olivia had her friend call her, to tell her th..."
2,018152fd-f984-4d05-ad1e-12f1fb7eceb6,Jack and Ferris always fought for headphones.,One day Jack broke Ferris' headphones while jo...,Ferris was furious at Jack.,Their parents yelled at them.,Jack promised Ferris to buy him new headphones.,Jack promised to take Ferris jogging.
3,feef76df-b75a-4501-9c1a-f8a7b6ee442f,I needed someone to help me move a bed across ...,"I called a couple of friends, but they were busy.","Finally, I called my grandson, who came right ...",He helped me move the bed to the right spot.,I made him an ice cream sundae for his efforts.,"He was tired from moving it, so he took a nap ..."
4,929eaf8b-a175-4460-a885-43be8a89ca62,Hannah had a beautiful cat that she loved very...,"However, she noticed that her cat was getting ...","One day, her cat ran away and never came back.",Hannah was devastated.,She never saw her beautiful cat again.,"The next day, she saw the cat walking down the..."


In [ ]:
print(ClozeTestSpring2016ValData.shape)
ClozeTestSpring2016ValData.head()

(1871, 8)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1
1,bff9f820-9605-4875-b9af-fe6f14d04256,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.,1
2,e8f628d5-9f97-40ed-8611-fc0e774673c4,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...,2
3,f5226bfe-9f26-4377-b05f-3d9568dbdec1,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.,1
4,69ac9b05-b956-402f-9fff-1f926ef9176b,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.,1


Please note that ClozeTestWinter2018TestData has no ground truth provided for us. We have to ask ROCStories and Story Cloze Test owner for grading. Therefore, for our final project evaluation, we may need to have another test dataset.

In [ ]:
pd.merge(ROCStoriesSpring2016, ROCStoriesWinter2017, how='inner').empty

True

In [ ]:
# In case some rows have different story ids but the same sentences
pd.merge(ROCStoriesSpring2016, ROCStoriesWinter2017, how='inner', on=["sentence1", "sentence2"]).empty

True

In [ ]:
pd.merge(ClozeTestSpring2016ValData, ClozeTestWinter2018ValData, how='inner', on=["InputSentence1", "InputSentence2"]).empty

False

We can see that two story cloze datasets have no overlap; we can have one as training and the other as test datasets. Two ROCStories datasets have no duplicate/overlap. We can safely combine them

In [ ]:
ROCStories = pd.concat([ROCStoriesSpring2016, ROCStoriesWinter2017])
ROCStories

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.
...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,The owner of the flavor sold him the recipe.
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,She breaks her wrist in the process and goes t...
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Jamie gets married and they spent the rest of ...


# Utility Functions

In [ ]:
def combine_sentences(row, sentence_columns_to_be_combined=["sentence1", "sentence2", "sentence3", "sentence4"]):
  assert len(sentence_columns_to_be_combined) > 0, "Sentence columns to be combined list length has to be larger than 0"
  combined_sentence = row[sentence_columns_to_be_combined[0]]
  for i in range(1, len(sentence_columns_to_be_combined)):
    combined_sentence += ' ' + row[sentence_columns_to_be_combined[i]]
  return combined_sentence

In [ ]:
def remove_first_four_sentences(paragraph):
    # Split the paragraph into sentences using regex
    sentences = re.split(r'(?<=[.!?])[\s"]+', paragraph)
    # Keep all sentences except the first 4
    remaining_sentences = sentences[4:]
    # Rejoin the sentences into a paragraph
    joined_sentences = ""
    for sentence in remaining_sentences:
      joined_sentences += sentence
      joined_sentences += " "
    joined_sentences = joined_sentences[:-1]
    return joined_sentences

def vectorized_remove_first_four_sentences(paragraphs):
    return np.vectorize(remove_first_four_sentences)(np.array(paragraphs))

In [ ]:
def concat_story_body_with_endings(story_body, story_ending):
  return [f"{p1} {p2}" for p1, p2 in zip(story_body, story_ending)]

# Train, Validation, Test Split

In [ ]:
ROCStories_Y = ROCStories["sentence5"]
ROCStories_X = ROCStories.drop(columns=["storyid", "storytitle", "sentence5"])
ROCStories_X = ROCStories_X.apply(combine_sentences, axis=1)
ROCStories_X

,0
0,Dan's parents were overweight. Dan was overwei...
1,Carrie had just learned how to ride a bike. Sh...
2,Morgan enjoyed long walks on the beach. She an...
3,"Jane was working at a diner. Suddenly, a custo..."
4,I was talking to my crush today. She continued...
...,...
52660,The man liked the flavor. He tried to recreate...
52661,"After my friend's dad's funeral, I got in trou..."
52662,Janice was out exercising for her big soccer g...
52663,Jamie is an american girl. Jamie wants to get ...


In [ ]:
ROCStories_X.shape

(98161,)

In [ ]:
ROCStories_Y.shape

(98161,)

We decided to combine some of ROCStories data and the Story Cloze 2018 Validation datasets as the final train datasets. The reasoning to additionally include Story Cloze dataset is that Story Cloze 2018 Validation datasets have the correct endings, therefore can be used for both classification or story ending generation evaluation. It also has a potential to be a bit different from the original ROCStories datasets that is mainly used as training data (in other words, increasing the variance of the test datasets for generalization).

In [ ]:
ROCX_train_val, ROCX_test, ROCy_train_val, ROCy_test = train_test_split(ROCStories_X, ROCStories_Y, test_size=0.02, random_state=seed_value)

In [ ]:
print(ROCX_train_val.shape)
print(ROCX_test.shape)
print(ROCy_train_val.shape)
print(ROCy_test.shape)

(96197,)
(1964,)
(96197,)
(1964,)


In [ ]:
Clozey_test = np.where(ClozeTestWinter2018ValData["AnswerRightEnding"]==1, ClozeTestWinter2018ValData["RandomFifthSentenceQuiz1"], ClozeTestWinter2018ValData["RandomFifthSentenceQuiz2"])
Clozey_test = pd.DataFrame(Clozey_test, columns=["sentence5"])
Clozey_test

,sentence5
0,He is happy now.
1,The brownies are so delicious Laverne eats two...
2,Sarah decided that she preferred her home over...
3,Gina liked the cookies so much she ate them al...
4,I was very proud of my performance.
...,...
1566,I have very fond memories of checkers.
1567,She loved her new phone.
1568,They were on sale.
1569,She was offered the new job at a higher salary.


In [ ]:
ClozeX_test = ClozeTestWinter2018ValData.drop(columns=["InputStoryid", "RandomFifthSentenceQuiz1", "RandomFifthSentenceQuiz2", "AnswerRightEnding"])
ClozeX_test = ClozeX_test.rename(columns={"InputSentence1": "sentence1", "InputSentence2": "sentence2", "InputSentence3": "sentence3", "InputSentence4": "sentence4"})
ClozeX_test = ClozeX_test.apply(combine_sentences, axis=1)
ClozeX_test

,0
0,Rick grew up in a troubled household. He never...
1,Laverne needs to prepare something for her fri...
2,Sarah had been dreaming of visiting Europe for...
3,Gina was worried the cookie dough in the tube ...
4,It was my final performance in marching band....
...,...
1566,When I was a kid I really wanted to play check...
1567,Ivy wanted a cell phone like all her friends. ...
1568,Dave walked into the grocery store. He was goi...
1569,Ramona was very unhappy in her job. She asked ...


In [ ]:
X_test = pd.concat([ROCX_test, ClozeX_test])
X_test

,0
20391,Evan had been saving for years. He went to the...
32389,Serena was planning a surprise for her husband...
8700,Fred slapped another man's butt. He thought he...
45048,I used to lived in Phoenix Arizona. On my way ...
23146,Tom thought he was really strong. He challenge...
...,...
1566,When I was a kid I really wanted to play check...
1567,Ivy wanted a cell phone like all her friends. ...
1568,Dave walked into the grocery store. He was goi...
1569,Ramona was very unhappy in her job. She asked ...


In [ ]:
y_test = pd.concat([ROCy_test, Clozey_test])
y_test

,sentence5
20391,Evan knew he looked cool in the new car.
32389,Together they were able to surprise him perfec...
8700,Fred apologized to the man.
45048,Talking with him not only made my day it made ...
23146,Tom's friends thought it was annoying.
...,...
1566,I have very fond memories of checkers.
1567,She loved her new phone.
1568,They were on sale.
1569,She was offered the new job at a higher salary.


In [ ]:
ROCX_train, ROCX_val, ROCy_train, ROCy_val = train_test_split(ROCX_train_val, ROCy_train_val, test_size=0.2, random_state=seed_value)

In [ ]:
ROCX_train = ROCX_train.to_list()
ROCX_val = ROCX_val.to_list()
ROCy_train = ROCy_train.to_list()
ROCy_val = ROCy_val.to_list()
X_test = X_test.to_list()
y_test = y_test["sentence5"].to_list()

In [ ]:
print(len(ROCX_train))
print(len(ROCX_val))
print(len(ROCy_train))
print(len(ROCy_val))
print(len(X_test))
print(len(y_test))

76957
19240
76957
19240
3535
3535


# Bart Model and Baseline

## Load BART Model

In [ ]:
model_name = "facebook/bart-large-cnn"
base_model = BartForConditionalGeneration.from_pretrained(model_name)
base_tokenizer = BartTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
base_model.to("cuda")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

## Prepare test data for evaluation

In [ ]:
# Tokenize the input text
X_test_inputs = base_tokenizer(X_test, padding=True, return_tensors="pt")
X_test_inputs

{'input_ids': tensor([[    0,   717,  9965,  ...,     1,     1,     1],
        [    0,   104,  2816,  ...,     1,     1,     1],
        [    0, 33153, 18361,  ...,     1,     1,     1],
        ...,
        [    0, 33857,  3203,  ...,     1,     1,     1],
        [    0, 32361,  4488,  ...,     1,     1,     1],
        [    0,   100,   770,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Move inputs to CUDA so that model.generate(**inputs) will use GPU. Once you do this, you can see GPU RAM increases and RAM (CPU RAM) is relatively stable. Otherwise, only CPU is used, which is slow, roughly taking 3 to 4 hours for generating ids for 3500ish test data.

In [ ]:
X_test_inputs["input_ids"] = X_test_inputs["input_ids"].to("cuda")
X_test_inputs["attention_mask"] = X_test_inputs["attention_mask"].to("cuda")

In [ ]:
X_test_inputs['input_ids'].device

device(type='cuda', index=0)

In [ ]:
X_test_inputs_dataloader = DataLoader(TensorDataset(X_test_inputs["input_ids"], X_test_inputs["attention_mask"]), batch_size=32)
test_original_story_with_endings_dataloader = DataLoader(concat_story_body_with_endings(X_test, y_test), batch_size=32)
original_story_bodies = DataLoader(X_test, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)

## Calculate perplexity, rouge, LLm as Judge score with basic BART as baseline

In [ ]:
judge_model = VLLM(model="prometheus-eval/prometheus-7b-v2.0", gpu_memory_utilization=0.5, enforce_eager=True, max_num_seqs=8, max_model_len=2048)
judge = PrometheusEval(model=judge_model, absolute_grade_template=ABSOLUTE_PROMPT)
rubric_data = {
  "criteria":"Does the model successfully generate a response that is a suitable ending to the provided story body, in terms of language fluency, semantics coherence, and story flow",
  "score1_description":"The ending is riddled with language errors, is incoherent or disconnected from the story body, and disrupts the narrative flow. It leaves the reader confused or unsatisfied.",
  "score2_description":"The ending has noticeable language issues, inconsistencies, or rushed transitions. While it ties some loose ends, it feels incomplete or awkward.",
  "score3_description":"The ending is adequately written with minor language errors and mostly logical progression, but it lacks emotional impact or creativity in its resolution.",
  "score4_description":"The ending is well-crafted, fluent, and coherent, with a fitting and satisfying resolution. It enhances the story’s themes and characters, though it might lack exceptional originality or depth.",
  "score5_description":"The ending is flawless in language fluency, beautifully integrates with the story, and delivers a compelling, imaginative, and emotionally resonant conclusion that elevates the entire narrative."
}
score_rubric = SCORE_RUBRIC_TEMPLATE.format(**rubric_data)

INFO 12-08 09:43:34 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
WARNING 12-08 09:43:34 config.py:503] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-08 09:43:34 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='prometheus-eval/prometheus-7b-v2.0', speculative_config=None, tokenizer='prometheus-eval/prometheus-7b-v2.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), ob

Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


INFO 12-08 09:43:40 model_runner.py:1077] Loading model weights took 13.4966 GB
INFO 12-08 09:43:40 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=15.52GiB peak_torch_memory=15.23GiB memory_usage_post_profile=15.54GiB non_torch_memory=0.52GiB kv_cache_size=4.03GiB gpu_memory_utilization=0.50
INFO 12-08 09:43:41 gpu_executor.py:113] # GPU blocks: 2064, # CPU blocks: 2048
INFO 12-08 09:43:41 gpu_executor.py:117] Maximum concurrency for 2048 tokens per request: 16.12x


In [ ]:
%%time
final_perplexity_baseline = 0
final_rouge1_baseline = 0
final_rouge2_baseline = 0
final_rougeL_baseline = 0
LLM_as_judge_score = 0

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for input_and_attention_mask, original_stories, reference in zip(X_test_inputs_dataloader, original_story_bodies, original_story_endings_reference):
    generated_ids = base_model.generate(inputs=input_and_attention_mask[0], attention_mask=input_and_attention_mask[1], min_new_tokens=70)
    full_story_with_ending_test = base_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    # Calculate perplexity
    weight_factor = len(reference) / len(X_test)
    final_perplexity_baseline += weight_factor * perplexity.compute(predictions=full_story_with_ending_test, model_id='facebook/bart-large-cnn')["mean_perplexity"]
    # Calculate rouge
    # Pop empty strings given in some rare cases, BART doesn't copy down the exact prompt, especially missing punctuations
    empty_indices = [i for i, s in enumerate(vectorized_remove_first_four_sentences(full_story_with_ending_test)) if s == ""]
    empty_indices.sort(reverse=True)
    for index in empty_indices:
      full_story_with_ending_test.pop(index)
      original_stories.pop(index)
      reference.pop(index)
    generated_endings = vectorized_remove_first_four_sentences(full_story_with_ending_test)
    rouge_results = rouge.compute(predictions=generated_endings, references=reference)
    final_rouge1_baseline += weight_factor * rouge_results["rouge1"]
    final_rouge2_baseline += weight_factor * rouge_results["rouge2"]
    final_rougeL_baseline += weight_factor * rouge_results["rougeL"]
    # LLM as judge
    feedback, score = judge.absolute_grade(
      instructions=original_stories,
      responses=generated_endings,
      rubric=score_rubric,
      reference_answers=reference
    )
    LLM_as_judge_score += sum(score)

  0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 31/31 [00:06<00:00,  4.68it/s, est. speed input: 2643.43 toks/s, output: 884.87 toks/s]


Processed 31/31 instances.


Finalizing: 100%|██████████| 31/31 [00:00<00:00, 10011.04it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 15/15 [00:03<00:00,  4.07it/s, est. speed input: 2291.86 toks/s, output: 669.60 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 9841.16it/s]


TypeError: 'int' object is not iterable

In [ ]:
# The above error is by a line LLM_as_judge_score = sum(LLM_as_judge_score) / len(X_test), which is wrong. The below is line is correct way:
LLM_as_judge_score = LLM_as_judge_score / len(X_test)

In [ ]:
print(final_perplexity_baseline)
print(final_rouge1_baseline)
print(final_rouge2_baseline)
print(final_rougeL_baseline)
print(LLM_as_judge_score)

502094.9014365276
0.13686567364345759
0.020479408512765515
0.11643881049054727
1.85007072135785


## Calculate perplexity score with original story and baseline model

In [ ]:
%%time
final_perplexity_original_story = 0
for reference in test_original_story_with_endings_dataloader:
  weight_factor = len(reference) / len(X_test)
  perplexity = evaluate.load("perplexity", module_type="metric")
  final_perplexity_original_story += weight_factor * perplexity.compute(predictions=reference, model_id='facebook/bart-large-cnn')["mean_perplexity"];

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1min 26s, sys: 8.68 s, total: 1min 35s
Wall time: 2min 31s


In [ ]:
final_perplexity_original_story

1384072.037551936

## LLM judge socre with original story

In [ ]:
%%time
LLM_judge_score_original_story = 0
for original_stories, reference in zip(original_story_bodies, original_story_endings_reference):
  feedback, score = judge.absolute_grade(
      instructions=original_stories,
      responses=reference,
      rubric=score_rubric,
      reference_answers=reference
    )
  LLM_judge_score_original_story += sum(score)
LLM_judge_score_original_story = LLM_judge_score_original_story/len(X_test)
LLM_judge_score_original_story

Processed prompts: 100%|██████████| 32/32 [00:20<00:00,  1.56it/s, est. speed input: 847.17 toks/s, output: 270.33 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:21<00:00,  1.49it/s, est. speed input: 813.53 toks/s, output: 261.96 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:19<00:00,  1.67it/s, est. speed input: 913.16 toks/s, output: 289.65 toks/s]


...

Processed prompts: 100%|██████████| 15/15 [00:08<00:00,  1.68it/s, est. speed input: 913.05 toks/s, output: 282.41 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 10275.12it/s]

CPU times: user 35min 18s, sys: 6.4 s, total: 35min 24s
Wall time: 35min 10s


3.551909476661952

## Check generated outputs by baseline

In [ ]:
X_test[0]

'Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town.'

In [ ]:
y_test[0]

'Evan knew he looked cool in the new car.'

As we can there are some slight difference between two returned candidates. Also some difference across different temperature. And if we set do_sample=True, even with same temperature, there is also some difference.

In [ ]:
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], temperature=0.5, do_sample=True, num_return_sequences=2), skip_special_tokens=True)

['Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. It was a big hit with his friends and family. He was so excited to have a new car to drive.',
 'Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. It was a big hit with his friends and family. He was so happy to have a new car to drive.']

In [ ]:
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], temperature=1.2, do_sample=True, num_return_sequences=2), skip_special_tokens=True)

['Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. He said it was the first car he had ever owned that was so fun to drive.',
 'Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. He said it was the first car he had ever owned. He had saved for years to get it.']

In [ ]:
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], temperature=1.2, do_sample=True, num_return_sequences=2), skip_special_tokens=True)

['Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. He said he was going to drive it all over the country. He was so excited to have a car of his own.',
 'Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. He said he was going to drive it all over the country. He was so excited to have a new car to drive.']

Now let's see an output withd default config.

In [ ]:
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1]), skip_special_tokens=True)

['Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. It was a big hit with his friends and family. He was so excited to have a new car to drive.']

In [ ]:
# Example of calling LLM as judge for a small subset of data

instructions = X_test[0:1]
responses = ["It was a big hit with his friends and family. He was so excited to have a new car to drive."]
reference_answers = y_test[0:1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.91s/it, est. speed input: 188.89 toks/s, output: 41.97 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 1885.93it/s]

Feedback: ['The response is adequate in terms of language fluency and semantics coherence, as it does not introduce any language errors and maintains the logical flow of the story. However, it falls short in delivering a compelling or imaginative resolution to the narrative. It merely states the fact that the new car was well-received without adding any emotional depth or creativity to the story. Therefore, while the response is well-crafted and coherent, it lacks the originality and depth that would elevate the entire narrative. So the overall score is 3. [RESULT] 3']
Score: [3]


Now try with other test data entries

In [ ]:
X_test[-2:-1]

['Ramona was very unhappy in her job. She asked for a raise, but was denied. The refusal prompted her to aggressively comb the want ads. She found an interesting new possibility and set up an interview.']

In [ ]:
y_test[-2]

'She was offered the new job at a higher salary.'

In [ ]:
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][-2:-1], attention_mask=X_test_inputs["attention_mask"][0:1]), skip_special_tokens=True)

['Ramona was very unhappy in her job. She asked for a raise, but was denied. The refusal prompted her to aggressively comb the want ads. She found an interesting new possibility and took it up. She was able to get a raise and a new job in the process.']

In [ ]:
# This random sample has the CNN.com
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][-2:-1], attention_mask=X_test_inputs["attention_mask"][0:1], do_sample=True), skip_special_tokens=True)

["Ramona was very unhappy in her job. She asked for a raise, but was denied. The refusal prompted her to aggressively comb the want ads. She found an interesting new possibility and took it on. Ramona's story will be featured on CNN.com this week."]

In [ ]:
instructions = X_test[-2:-1]
responses = ["She was able to get a raise and a new job in the process."]
reference_answers = y_test[-2:-1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.50s/it, est. speed input: 121.78 toks/s, output: 42.22 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 4429.04it/s]

Feedback: ["This response, while satisfactorily wrapping up the story, does not exhibit the same level of detail and creativity seen in the reference response. It simply states that Ramona received a raise and a new job, without specifying what made the new job interesting or how the interview process unfolded. This lack of elaboration results in a somewhat flat ending that doesn't fully capture the dramatic and emotional journey that Ramona embarked on. Additionally, the language used is straightforward and unadorned, which doesn't necessarily enhance the story's themes and characters. On the other hand, the response maintains coherence and does not disrupt the narrative flow, thereby meeting some criteria outlined in the score rubric. Therefore, while it is not the most captivating or emotionally resonant ending, it is still adequate and coherent. So the overall score is 3. \n[RESULT] 3"]
Score: [3]


In [ ]:
print(X_test[1500:1501])
print(y_test[1500:1501])
base_tokenizer.batch_decode(base_model.generate(X_test_inputs["input_ids"][1500:1501], attention_mask=X_test_inputs["attention_mask"][0:1]), skip_special_tokens=True)

['I went to the doctor yesterday today for a check up. The doctor told me I needed a shot. When I got the shot I screamed. The pain was unbearable.']
['The only upside was that the pain was quick.']


["I went to the doctor yesterday today for a check up. The doctor told me I needed a shot. When I got the shot I screamed. The pain was unbearable. I was in so much pain that I had to go home and take a nap. I'm still in a lot of pain."]

In [ ]:
instructions = X_test[1500:1501]
responses = ["I was in so much pain that I had to go home and take a nap. I'm still in a lot of pain."]
reference_answers = y_test[1500:1501]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.28s/it, est. speed input: 128.58 toks/s, output: 41.38 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5745.62it/s]

Feedback: ["The response effectively continues the storyline, however, it falls short in providing an emotionally resonant conclusion. It relies heavily on a singular aspect of the narrative – the pain – without incorporating any creative or imaginative elements. Additionally, the ending is somewhat abrupt and does not convey a strong sense of satisfaction or closure to the story. Despite this, the language used is clear and free from errors, and it maintains coherence with the story's semantics. However, it fails to enhance the story’s themes or characters. In terms of story flow, the transition from the doctor's advice to the protagonist's reaction at home is somewhat disjointed, but it does not disrupt the overall narrative. Therefore, the response is deemed adequate, but with room for improvement in emotional resonance and creativity. \n[RESULT] 3"]
Score: [3]


## [Archived] Calculate perplexity and rouge with only a subset of test datasets with size=32

It's very memory and time consuming to do generate, 32GB ram crashes and we have to batch. Let's use only a small subset of test data for now.

In [ ]:
%%time
# Generate the story ending
with torch.no_grad():
  generated_ids = base_model.generate(X_test_inputs["input_ids"][:32], min_new_tokens=70) #Set min_new_tokens to ensure an ending is generated
generated_ids

CPU times: user 1min 54s, sys: 14.2 s, total: 2min 8s
Wall time: 2min 9s


tensor([[    2,     0,   717,  ...,     1,     1,     1],
        [    2,     0,   104,  ...,     1,     1,     1],
        [    2,     0, 33153,  ...,     1,     1,     1],
        ...,
        [    2,     0, 18031,  ...,     1,     1,     1],
        [    2,     0, 23675,  ...,     1,     1,     1],
        [    2,     0, 24021,  ...,     1,     1,     1]])

In [ ]:
# Decode the generated ending
# Please note that currently generated text includes both the input, story beginning, and the generated ending
full_story_with_ending_test = base_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
full_story_with_ending_test[0]

"Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. It was a big hit with his friends and family. He was so excited to have a new car to drive around in. It's been a big success. He's so proud."

In [ ]:
perplexity = evaluate.load("perplexity", module_type="metric")
perplexity_results = perplexity.compute(predictions=full_story_with_ending_test, model_id='facebook/bart-large-cnn')

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
perplexity_results["mean_perplexity"]

208850.79104614258

Let's also see the original story's perplexity score

In [ ]:
original_story_with_endings_test = [f"{p1} {p2}" for p1, p2 in zip(X_test, y_test)]
original_story_with_endings_test[0]

'Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. Evan knew he looked cool in the new car.'

In [ ]:
perplexity.compute(predictions=original_story_with_endings_test[:32], model_id='facebook/bart-large-cnn')["mean_perplexity"]

  0%|          | 0/2 [00:00<?, ?it/s]

580600.5565185547

Now also see Rouge. We use Rouge our problem is more like a summarization, more abstractive and open ended; therefore, when comparing to the ereference, we care more about how much reference info has been captured rather than if the whole generated text is strictly alike the reference.

In [ ]:
empty_indices = [i for i, s in enumerate(vectorized_remove_first_four_sentences(full_story_with_ending_test)) if s == ""]
empty_indices

[20]

In [ ]:
X_test[20]

'Bella came to school with an unexpected present on her desk. To her surprise, she was given a bouquet of beautiful flowers. Bella took a look at the note and it said "Secret Admirer". She took a sniff of the flowers as they smelled wonderful.'

In [ ]:
full_story_with_ending_test[20]

'Bella came to school with an unexpected present on her desk. To her surprise, she was given a bouquet of beautiful flowers. Bella took a look at the note and it said "Secret Admirer" She took a sniff of the flowers as they smelled wonderful. They were from her secret admirer, who she has never met.'

In [ ]:
full_story_with_ending_test.pop(20)

'Bella came to school with an unexpected present on her desk. To her surprise, she was given a bouquet of beautiful flowers. Bella took a look at the note and it said "Secret Admirer" She took a sniff of the flowers as they smelled wonderful. They were from her secret admirer, who she has never met.'

In [ ]:
empty_indices = [i for i, s in enumerate(vectorized_remove_first_four_sentences(full_story_with_ending_test)) if s == ""]
empty_indices

[]

In [ ]:
ending_ref_test = y_test[:32]
ending_ref_test.pop(20)
len(ending_ref_test)

31

In [ ]:
rouge = evaluate.load('rouge')
rouge_results = rouge.compute(predictions=vectorized_remove_first_four_sentences(full_story_with_ending_test), references=ending_ref_test)

In [ ]:
rouge_results

{'rouge1': 0.13083555512743467,
 'rouge2': 0.0238949919279419,
 'rougeL': 0.11233191004109086,
 'rougeLsum': 0.11360697325944744}

# Bart Model Fine Tuning

## Fine Tuning BART

In [ ]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
train_encodings = tokenizer(ROCX_train, padding=True)
val_encodings = tokenizer(ROCX_val, padding=True)

In [ ]:
train_labels = tokenizer(ROCy_train, padding=True, return_tensors="pt")['input_ids']
val_labels = tokenizer(ROCy_val, padding=True, return_tensors="pt")['input_ids']

In [ ]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [ ]:
training_args = TrainingArguments(
    report_to="none",
    output_dir='./fine_tuned_bart_checkpoints',
    save_strategy="steps",
    eval_strategy="steps",     # Evaluation frequency
    save_steps=2000,
    eval_steps=2000,
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size per GPU for training
    per_device_eval_batch_size=16,   # Batch size per GPU for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
%%time
trainer.train()

Step,Training Loss,Validation Loss
2000,1.245700,1.206727
4000,1.211600,1.165745
6000,1.191200,1.168289
8000,1.167800,1.172269
10000,0.963200,1.179446
12000,0.889400,1.126464
14000,0.887500,1.130638
16000,0.889900,1.109606
18000,0.882800,1.143582
20000,0.626800,1.249945


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


CPU times: user 1h 17min 55s, sys: 1min 40s, total: 1h 19min 35s
Wall time: 1h 20min 35s


TrainOutput(global_step=28860, training_loss=0.9068856665597388, metrics={'train_runtime': 4834.8855, 'train_samples_per_second': 47.751, 'train_steps_per_second': 5.969, 'total_flos': 4.250779274774938e+16, 'train_loss': 0.9068856665597388, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./drive/MyDrive/fine_tuned_bart")

## Evaluate fine tuned BART model

In [ ]:
fine_tuned_model_path = "./drive/MyDrive/fine_tuned_bart"

# The model loading doesn't seem to work
#fine_tuned_model = BartForConditionalGeneration.from_pretrained(fine_tuned_model_path, revision="safetensors")
fine_tuned_model=model

In [ ]:
fine_tuned_model.to("cuda")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
%%time
final_perplexity_baseline = 0
final_rouge1_baseline = 0
final_rouge2_baseline = 0
final_rougeL_baseline = 0

inputs_to_be_generated_data = X_test
inputs_to_be_generated_dataloader = DataLoader(inputs_to_be_generated_data, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)
model_to_use = fine_tuned_model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for inputs_to_be_generated, reference in zip(inputs_to_be_generated_dataloader, original_story_endings_reference):
    input_and_attention_mask = tokenizer(inputs_to_be_generated, padding=True, return_tensors="pt")
    input_and_attention_mask.to("cuda")
    generated_ids = model_to_use.generate(inputs=input_and_attention_mask["input_ids"], attention_mask=input_and_attention_mask["attention_mask"], max_length=20, early_stopping=True)
    ending = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    full_story_with_ending = concat_story_body_with_endings(inputs_to_be_generated, ending)
    # Calculate perplexity
    weight_factor = len(reference) / len(inputs_to_be_generated_data)
    final_perplexity_baseline += weight_factor * perplexity.compute(predictions=full_story_with_ending, model_id='facebook/bart-large-cnn')["mean_perplexity"]
    # Calculate rouge
    rouge_results = rouge.compute(predictions=ending, references=reference)
    final_rouge1_baseline += weight_factor * rouge_results["rouge1"]
    final_rouge2_baseline += weight_factor * rouge_results["rouge2"]
    final_rougeL_baseline += weight_factor * rouge_results["rougeL"]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 3min 42s, sys: 8.91 s, total: 3min 51s
Wall time: 4min 41s


In [ ]:
print(final_perplexity_baseline) #epoch 3
print(final_rouge1_baseline)
print(final_rouge2_baseline)
print(final_rougeL_baseline)

1100737.7915157855
0.22182885500760363
0.05309287064959933
0.19703626011511763


In [ ]:
%%time
LLM_as_judge_score_fine_tuned = 0

X_test_inputs_dataloader = DataLoader(TensorDataset(X_test_inputs["input_ids"], X_test_inputs["attention_mask"]), batch_size=32)
original_story_bodies = DataLoader(X_test, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)
model_to_use = fine_tuned_model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for input_and_attention_mask, original_stories, reference in zip(X_test_inputs_dataloader, original_story_bodies, original_story_endings_reference):
    generated_ids = model_to_use.generate(inputs=input_and_attention_mask[0], attention_mask=input_and_attention_mask[1], max_length=20, early_stopping=True)
    generated_endings = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    feedback, score = judge.absolute_grade(
      instructions=original_stories,
      responses=generated_endings,
      rubric=score_rubric,
      reference_answers=reference
    )
    LLM_as_judge_score_fine_tuned += sum(score)
LLM_as_judge_score_fine_tuned = LLM_as_judge_score_fine_tuned/len(X_test)
LLM_as_judge_score_fine_tuned

Processed prompts: 100%|██████████| 32/32 [00:19<00:00,  1.60it/s, est. speed input: 879.15 toks/s, output: 286.34 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:19<00:00,  1.61it/s, est. speed input: 881.72 toks/s, output: 288.92 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:18<00:00,  1.75it/s, est. speed input: 965.92 toks/s, output: 291.75 toks/s]


...

Processed prompts: 100%|██████████| 15/15 [00:09<00:00,  1.57it/s, est. speed input: 860.84 toks/s, output: 256.70 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 7637.12it/s]

CPU times: user 38min 5s, sys: 5.29 s, total: 38min 10s
Wall time: 37min 59s


2.039321074964639

In [ ]:
LLM_as_judge_score_fine_tuned

2.039321074964639

In [ ]:
instructions = X_test[0:1]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[0:1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

['Evan was so happy with his new car.   car!   ']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.76s/it, est. speed input: 143.85 toks/s, output: 43.69 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5349.88it/s]

Feedback: ["The generated response is an attempt to capture the excitement of Evan regarding his new car. However, the lack of proper punctuation and sentence structure makes it difficult to read and could be interpreted as an exclamation about the car itself, rather than Evan's happiness. Additionally, the brevity of the response lacks depth and fails to provide a satisfying conclusion to the story, which could potentially leave readers feeling unfulfilled. It also disrupts the narrative flow by not directly addressing Evan's actions and feelings towards the end. Thus, while the response attempts to reflect the initial instruction, it falls short of meeting the expectations of a well-crafted ending that is fluent, coherent, and emotionally resonant. So the overall score is 2. [RESULT] 2"]
Score: [2]


In [ ]:
instructions = X_test[1500:1501]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][1500:1501], attention_mask=X_test_inputs["attention_mask"][1500:1501], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[1500:1501]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

['I had to go home and rest for the rest of the day. yelp']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.53s/it, est. speed input: 152.92 toks/s, output: 43.33 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5637.51it/s]

Feedback: ['The response provides a continuation of the story but does so in a manner that is disjointed from the original narrative. It fails to capture the gravity of the situation presented in the instruction and does not demonstrate a coherent understanding of the emotional turmoil experienced by the protagonist. Furthermore, the use of "yelp" at the end appears as an abrupt and irrelevant interjection that does not contribute to the story\'s progression. The lack of context, coupled with the abrupt ending, disrupts the narrative flow and leaves the reader unsatisfied. This response fails to meet the standards of language fluency, semantics coherence, and story flow as required by the score rubric. \n[RESULT] 1']
Score: [1]


In [ ]:
instructions = X_test[-2:-1]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][-2:-1], attention_mask=X_test_inputs["attention_mask"][-2:-1], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[-2:-1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

['Ramona was offered the new job and was very happy with it, after all.']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.37s/it, est. speed input: 163.33 toks/s, output: 42.98 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5562.74it/s]

Feedback: ["The response successfully concludes the story with the intended outcome. Ramona finds happiness in a new job, which aligns with the story's arc. However, the language used is too simplistic and does not provide a strong emotional resonance with the reader. The response also lacks a creative or imaginative element that could have further enriched the narrative. Despite these shortcomings, the conclusion is logically connected to the story, ensuring that the reader is left satisfied. Therefore, while the response meets the basic requirements for story flow and coherence, it falls short in providing a compelling ending with rich language and emotional depth. Hence, the overall score is 3. [RESULT] 3"]
Score: [3]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, do_sample=False), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=0.8, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=0.3, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=1.5, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=5, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=20, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=80, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.8, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.5, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()

for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.3, do_sample=True), skip_special_tokens=True))
  print()
  print("-----------------------------------------------------------------------")
  print()


["Evan's friends and family loved his new car, and he was happy. "]

-----------------------------------------------------------------------

['Evan was so happy with his new car and was excited to drive it home.']

-----------------------------------------------------------------------

["Evan was so happy with his new car and couldn't wait to drive it."]

-----------------------------------------------------------------------

['Evan was very happy with his new car.   it was a new model']

-----------------------------------------------------------------------

['Evan was so happy with his new car.    new car and was']

-----------------------------------------------------------------------

['Evan was so happy with his new car!   car is his favorite!']

-----------------------------------------------------------------------

['Evan was so happy with his new car.   car was very expensive.']

-----------------------------------------------------------------------

['Evan was so happy 

## Further fine tune the BART Model

Previously the best model was selected around 19000th step, which is roughly 2 epoch (I deduced this by looking at the training loss, around 0.7, when continuing fine tuning, at the time when I didn't change the per_device_train_batch_size yet).

In [ ]:
training_args = TrainingArguments(
    report_to="none",
    output_dir='./fine_tuned_bart_checkpoints',
    save_strategy="steps",
    eval_strategy="steps",     # Evaluation frequency
    save_steps=2000,
    eval_steps=2000,
    save_total_limit=1,
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=32,   # Batch size per GPU for training
    per_device_eval_batch_size=64,   # Batch size per GPU for evaluation
    num_train_epochs=6,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [ ]:
%%time
trainer.train()

Step,Training Loss,Validation Loss
2000,0.739300,1.244433
4000,0.477600,1.419745
6000,0.334300,1.604613
8000,0.213900,1.704297
10000,0.156400,1.926931
12000,0.137000,1.926873
14000,0.087700,2.039231


CPU times: user 1h 18min 45s, sys: 3min 34s, total: 1h 22min 19s
Wall time: 1h 22min 4s


TrainOutput(global_step=14430, training_loss=0.321843570648426, metrics={'train_runtime': 4923.5888, 'train_samples_per_second': 93.782, 'train_steps_per_second': 2.931, 'total_flos': 8.501558549549875e+16, 'train_loss': 0.321843570648426, 'epoch': 6.0})

From the validation loss, we can see increasing epoch doesn't do anything good, therefore the previous model trained with around 2 epochs is our selected model. Just out of curiosity, to see that validation loss indeed reflects model performance, I still apply evaluation metrics over test data so that we can eyeball the generated output and have a glimpse on the metrics. However, please understand that by no means we are using the metrics from test data for model selection.

In [ ]:
fine_tuned_model=model
fine_tuned_model.to("cuda")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
%%time
final_perplexity_baseline = 0
final_rouge1_baseline = 0
final_rouge2_baseline = 0
final_rougeL_baseline = 0

inputs_to_be_generated_data = X_test
inputs_to_be_generated_dataloader = DataLoader(inputs_to_be_generated_data, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)
model_to_use = fine_tuned_model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for inputs_to_be_generated, reference in zip(inputs_to_be_generated_dataloader, original_story_endings_reference):
    input_and_attention_mask = tokenizer(inputs_to_be_generated, padding=True, return_tensors="pt")
    input_and_attention_mask.to("cuda")
    generated_ids = model_to_use.generate(inputs=input_and_attention_mask["input_ids"], attention_mask=input_and_attention_mask["attention_mask"], max_length=20, early_stopping=True)
    ending = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    full_story_with_ending = concat_story_body_with_endings(inputs_to_be_generated, ending)
    # Calculate perplexity
    weight_factor = len(reference) / len(inputs_to_be_generated_data)
    final_perplexity_baseline += weight_factor * perplexity.compute(predictions=full_story_with_ending, model_id='facebook/bart-large-cnn')["mean_perplexity"]
    # Calculate rouge
    rouge_results = rouge.compute(predictions=ending, references=reference)
    final_rouge1_baseline += weight_factor * rouge_results["rouge1"]
    final_rouge2_baseline += weight_factor * rouge_results["rouge2"]
    final_rougeL_baseline += weight_factor * rouge_results["rougeL"]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 3min 41s, sys: 8.87 s, total: 3min 50s
Wall time: 4min 20s


In [ ]:
print(final_perplexity_baseline)
print(final_rouge1_baseline)
print(final_rouge2_baseline)
print(final_rougeL_baseline)

1065652.2555499696
0.20002404282228245
0.04110460788753432
0.1759953809377672


In [ ]:
%%time
LLM_as_judge_score_fine_tuned = 0

X_test_inputs_dataloader = DataLoader(TensorDataset(X_test_inputs["input_ids"], X_test_inputs["attention_mask"]), batch_size=32)
original_story_bodies = DataLoader(X_test, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)
model_to_use = fine_tuned_model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for input_and_attention_mask, original_stories, reference in zip(X_test_inputs_dataloader, original_story_bodies, original_story_endings_reference):
    generated_ids = model_to_use.generate(inputs=input_and_attention_mask[0], attention_mask=input_and_attention_mask[1], max_length=20, early_stopping=True)
    generated_endings = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    feedback, score = judge.absolute_grade(
      instructions=original_stories,
      responses=generated_endings,
      rubric=score_rubric,
      reference_answers=reference
    )
    LLM_as_judge_score_fine_tuned += sum(score)
LLM_as_judge_score_fine_tuned = LLM_as_judge_score_fine_tuned/len(X_test)
LLM_as_judge_score_fine_tuned

Processed prompts: 100%|██████████| 62/62 [00:34<00:00,  1.80it/s, est. speed input: 988.24 toks/s, output: 309.14 toks/s]


Processed 62/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:18<00:00,  1.70it/s, est. speed input: 934.14 toks/s, output: 283.96 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:19<00:00,  1.63it/s, est. speed input: 896.89 toks/s, output: 288.14 toks/s]


...

Processed prompts: 100%|██████████| 15/15 [00:10<00:00,  1.49it/s, est. speed input: 816.22 toks/s, output: 249.79 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 3877.39it/s]

CPU times: user 39min 48s, sys: 7.83 s, total: 39min 56s
Wall time: 39min 40s


1.8373408769448374

In [ ]:
instructions = X_test[0:1]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[0:1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

['All of his friends complimented him on his new car. krune/k']


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.08s/it, est. speed input: 132.75 toks/s, output: 42.62 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5769.33it/s]

Feedback: ['The provided response does not meet the expectations of the rubric. It presents a sentence that is disjointed and incoherent with the rest of the story. The language error "krune/k" disrupts the narrative flow and confuses the reader. This sudden and unexplained change in vocabulary does not contribute to a satisfactory conclusion. Furthermore, the sentence lacks the necessary elements to make it a suitable ending, such as detailing Evan\'s emotions or reactions, which would have enhanced the story\'s themes and characters. Overall, the response fails to tie loose ends or provide an imaginative or emotionally resonant conclusion to the story. It does not fulfill the criteria for language fluency, semantics coherence, and story flow, as specified in the score rubric. \n[RESULT] 1']
Score: [1]


In [ ]:
instructions = X_test[1500:1501]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][1500:1501], attention_mask=X_test_inputs["attention_mask"][1500:1501], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[1500:1501]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

['I then proceeded to scream for the rest of the visit. B. Belly B']


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.60s/it, est. speed input: 117.76 toks/s, output: 42.88 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 4369.07it/s]

Feedback: ['The given response falls short of the expectations outlined in the score rubric. Firstly, it lacks language fluency as it does not construct a coherent sentence. The structure "B. Belly B" seems to be a contraction of "Belly B" with a mention of "scream" in the middle, which is neither grammatically correct nor contextually appropriate. This abruptness disrupts the narrative flow, making the reader confused. Additionally, the response does not offer a satisfying resolution to the story, which was expected as per the instruction. The pain and the subsequent scream, while integral to the story, are not developed further in the response. Therefore, the ending feels incomplete and awkward. Based on these aspects, the response fails to meet the criteria of the score rubric for language fluency, semantics coherence, and story flow, hence, it receives a score of 1. \n[RESULT] 1']
Score: [1]


In [ ]:
instructions = X_test[-2:-1]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][-2:-1], attention_mask=X_test_inputs["attention_mask"][-2:-1], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[-2:-1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

['After the interview, the company offered Ramona a raise and she accepted! \\Pri']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.46s/it, est. speed input: 159.48 toks/s, output: 42.55 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5667.98it/s]

Feedback: ["This response offers a clear and positive resolution to Ramona's story, indicating that she received the raise she desired from her new employer. However, the response could have been more detailed and elaborative in terms of how the situation unfolded after the interview. This omission slightly impacts the depth of the narrative and the emotional resonance of the conclusion. Additionally, while the language is clear and free of errors, the response lacks a certain level of creativity in storytelling. Nevertheless, the overall flow and coherence of the story are maintained, making the conclusion satisfactory. Thus, while this response is competent, it falls short of achieving exceptional originality or depth. \n[RESULT] 3"]
Score: [3]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True), skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


["Evan couldn't wait for people to see his new car! \\¢"]
["Evan was so happy he'd saved so much money for the new car.\\"]
["Evan couldn't wait for people to see his new car! krinkle."]
['A neighbor even said he looked just like Evan! k. s. k']
['People were amazed and loved his new car. kr. k. k']
["Evan couldn't wait for everyone to see his new car! k.r"]
['All his friends and family are impressed by his new wheels. kludger.']
["Evan couldn't wait for people to see his new car! \\¢"]
['All his friends and family were jealous of the new car he had bought/ bought.']
["Evan couldn't wait for people to see the new car!\\\\"]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, do_sample=False), skip_special_tokens=True))

["Evan was glad he'd waited so long to buy the BMW. kludge"]
["Evan couldn't wait for everyone to see his brand new car! kr."]
['Evan was so excited to have a new car. k. s. kl']
["Evan couldn't wait for people to see his new car! k. s."]
['Evan was glad he had waited so long to buy the fancy car. k.']
['All his friends were impressed with his new wheels. kludge klod.']
['A few people thought it was a new car, and bought a pretty one.\\']
['A lot of people complimented him on his new ride. k. s. k']
["Evan couldn't wait for people to see his new car! klans b"]
['Evan was glad he had waited so long to buy his new car! \\Pri']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=0.8, do_sample=True), skip_special_tokens=True))

['A neighbor even offered to drive him home in his new blouse! kB.']
['Evan felt very accomplished to have a car that looked like a Lamborghini.']
['A lot of people were impressed with the new wheels. klaspy kl']
["Evan couldn't believe he'd finally saved enough to buy a new car!\\"]
["Evan was so happy he hadn't expected to be so proud!\\\\"]
["Evan couldn't wait for people to see the new car. k. s."]
['All of his friends thought it was the most beautiful car they had ever seen. \\']
["Evan couldn't wait for everyone to see his new car!\\Bundert"]
["Evan was glad he'd spent so much money on the new car. k."]
["Evan was so happy, he didn't care how old it was even though it"]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=0.3, do_sample=True), skip_special_tokens=True))

['Evan was glad he had waited so long to buy the car. k. k']
['All the other residents thought it was a new thing to see! k.d.']
["Evan couldn't wait for all his friends to see the new car! k."]
['Evan was so happy he had waited so long for the car!Prize!']
['A lot of people drove by and thought it was very nice.  It was a']
['A lot of people showed up to see how fancy it was. k. s.']
['A lot of people were impressed with his new wheels! k. s b. k']
['People were amazed at his beauty and he was very proud too. krr.']
["Evan couldn't wait for people to see the new car! kranks k"]
["Evan couldn't wait for people to see the new wheels!\\tB"]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=1.5, do_sample=True), skip_special_tokens=True))

["Evan knew people looked at it differently than any other car he'd owned.\\"]
["People were so impressed they couldn't recognize it was a new car. \\Derek"]
['When it was time for dinner, Evan and his friends feigned illness.Burg']
['A famous photographer even snapped a photo. s he got in his new car. k']
["Evan couldn't believe people didn't know it was worth hundreds of thousands.\\"]
['Evan was glad he had waited so long for his new car! k/k']
['A lot of people thought it was a new trend and joined in on the fun!']
["Evan couldn't believe how beautiful it was! kimim bakes ketchup"]
['A lot of people were impressed with his new wheels. k.e. kl']
['A few weeks later, the city took his picture and posted it on FB. k']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=5, do_sample=True), skip_special_tokens=True))

['All his friends gushed over how beautiful it was. s. k. t.']
["Evan couldn't wait for everyone to see his new wheels! B.B."]
['Evan was glad he had waited so long to buy his new car!\\t']
["Evan couldn't wait for people to see his new car!  New York Times"]
['A lot of people commented on the car and said it was very nice. k.']
['Evan was so thrilled with his new car! k.eepley. k']
['Evan wanted to get his girlfriend to buy the car with him! Bump,']
["Evan was happy he'd saved enough to buy such a fancy car. \\¢"]
['All his friends and family were impressed with the new wheels!\\MAYONAY']
['All of his friends and family were impressed with his new ride.\\d BSO']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=20, do_sample=True), skip_special_tokens=True))

["Evan was glad he'd waited so long to buy the new car. k."]
['A few weeks later, he sold it for a five thousand dollar profit! \\K']
['All his friends and girlfriend thought it was a great new car. kranks k']
['A few weeks later, it was sold to someone else for a lot of money.']
["Evan couldn't wait for everyone to see his new wheels! klaspal"]
['All the other residents wanted to take a ride in his new car! kelle']
["Evan couldn't wait for people to see his new car!\\tAAA"]
['All his friends and family fell in love with the new car Evan had bought.\\']
['Evan was so happy he had waited so long to buy the car! k.']
['A lot of people complimented him on his new car. k. klondon']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=80, do_sample=True), skip_special_tokens=True))

["Evan was glad he'd waited so long for the new car! B.B"]
["Evan was glad he'd waited so long to buy his dream car!--K"]
['A few weeks later, the dealership approved his new car and he was so happy.']
['All of his friends thought it was a new piece of art! k. s.']
['Evan was glad he had waited so long to buy his dream car. \\/']
['All the other residents thought it was a new style craze! k. k']
["People were amazed at how expensive it was before Evan's car was sold. Evan was"]
['All of his friends thought he was a real fashion model! kludge kl']
['All his friends thought it was a new car and wanted to buy one, too!']
["Evan couldn't wait for people to see his new wheels! k. d."]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.8, do_sample=True), skip_special_tokens=True))

["Evan couldn't wait for everyone to see his new car! k.d."]
['A neighbor even offered to buy it for him, it was such a nice car!']
["Evan couldn't wait for everyone to see his new car! \\ k"]
['All the girls in town wanted him to buy a new car. k. s.']
["Evan couldn't wait for all his friends to see it! k. s."]
["Evan couldn't wait for everyone to see the new wheels he'd bought!\\"]
["A few days later, someone bought Evan's car for $50,000! k"]
['People were so impressed they took him out to a fancy restaurant! \\SURPR']
['All of his friends and family complimented him on his new ride. \\B']
['Evan was so happy he had waited so long to buy the car. \\D']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.5, do_sample=True), skip_special_tokens=True))

["Evan couldn't wait for people to see his brand new car! kr."]
['A few weeks later, someone else bought the same car for a lot of money.']
["Evan couldn't wait for people to see his new car!\\\\"]
['Evan felt like he could walk on the clouds with his new car. \\D']
["Evan was glad he'd saved for this kind of car!\\Blessing"]
['Evan felt like a new person!  . . kludge klunch']
['Evan was sure no one would have a chance to appraise it. \\t']
['All the other people in town complimented him on his style. k. s.']
['People were amazed and said he must have a lot of money. \\B']
["Evan couldn't wait for all his friends to see it! \\�"]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.3, do_sample=True), skip_special_tokens=True))

['A lot of people thought he was a real jerk. k. k']
["Evan was glad he'd waited so long to get his new car!\\ k"]
["Evan couldn't wait for people to see his new wheels! k. s."]
['All of his friends and family complimented him on his new car. kludge']
["Evan couldn't wait for people to see his new car! k. kl"]
['A few weeks later, his insurance went up by a foot. B.B.']
['Evan was so happy he had waited so long to buy his dream car!\\']
['All his friends and family were jealous of the new car he had bought. \\D']
['All his friends thought it was a great new car. kranks krune']
["Evan couldn't wait for everyone to see his new car!\\ k"]


## peft with LORA
It seems that with A100 GPU, lora doesn't make too much of imporvement, still around 1hr. But with Colab T4, lora cuts the training time from 4hr to 2hr and 20 minutes ish.

In [ ]:
from peft import get_peft_model, LoraConfig
from peft import TaskType

# Set up LoRA configuration
lora_config = LoraConfig(
    r=4,              # Rank of the low-rank matrices
    lora_alpha=8,    # Scaling factor for the low-rank adaptation
    lora_dropout=0.1, # Dropout rate for LoRA layers
    task_type=TaskType.SEQ_2_SEQ_LM  # Task type for sequence-to-sequence model
)

# Integrate LoRA with the model
lora_model = get_peft_model(model, lora_config)

In [ ]:
lora_trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
lora_trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model("./fine_tuned_bart_lora")

# Further Fine Tune the Encoder with Story Cloze

In [ ]:
ClozeTestSpring2016ValData.head()

,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1
1,bff9f820-9605-4875-b9af-fe6f14d04256,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.,1
2,e8f628d5-9f97-40ed-8611-fc0e774673c4,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...,2
3,f5226bfe-9f26-4377-b05f-3d9568dbdec1,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.,1
4,69ac9b05-b956-402f-9fff-1f926ef9176b,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.,1


In [ ]:
ClozeTestSpring2016ValData["Story1"] = combine_sentences(ClozeTestSpring2016ValData, sentence_columns_to_be_combined=["InputSentence1", "InputSentence2", "InputSentence3", "InputSentence4", "RandomFifthSentenceQuiz1"])[0]
ClozeTestSpring2016ValData["Story2"] = combine_sentences(ClozeTestSpring2016ValData, sentence_columns_to_be_combined=["InputSentence1", "InputSentence2", "InputSentence3", "InputSentence4", "RandomFifthSentenceQuiz2"])[0]

In [ ]:
ClozeTestTraining = ClozeTestSpring2016ValData.drop(columns=["InputSentence1", "InputSentence2", "InputSentence3", "InputSentence4", "RandomFifthSentenceQuiz1", "RandomFifthSentenceQuiz2", "InputStoryid"])

In [ ]:
ClozeTestTraining["AnswerRightEnding"] = ClozeTestTraining["AnswerRightEnding"] - 1
ClozeTestTraining

,AnswerRightEnding,Story1,Story2
0,0,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
1,0,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
2,1,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
3,0,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
4,0,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
...,...,...,...
1866,1,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
1867,1,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
1868,1,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...
1869,1,Rick grew up in a troubled household. He never...,Rick grew up in a troubled household. He never...


In [ ]:
# Freeze all decoder parameters so that only encoder parameters are updated during training
for param in model.model.decoder.parameters():
    param.requires_grad = False

In [ ]:
import torch
import torch.nn as nn

class BartEncoderForClassification(nn.Module):
    def __init__(self, bart_model):
        super().__init__()
        self.bart = bart_model  # This is the BartForConditionalGeneration model
        self.fc1 = nn.Linear(2048, 1)
        self.classifier = nn.Sigmoid()

    def forward(self, input_ids_1, attention_mask_1, input_ids_2, attention_mask_2, labels=None):
        # Encode the first input
        encoder_outputs_1 = self.bart.model.encoder(
            input_ids=input_ids_1,
            attention_mask=attention_mask_1
        )
        last_hidden_state_1 = encoder_outputs_1[0]  # (batch_size, seq_len, hidden_dim)
        pooled_output_1 = last_hidden_state_1[:, 0, :]  # Take the first token as representation

        # Encode the second input
        encoder_outputs_2 = self.bart.model.encoder(
            input_ids=input_ids_2,
            attention_mask=attention_mask_2
        )
        last_hidden_state_2 = encoder_outputs_2[0]
        pooled_output_2 = last_hidden_state_2[:, 0, :]

        # Combine both representations
        combined_output = torch.cat([pooled_output_1, pooled_output_2], dim=-1)
        logits = self.fc1(combined_output)
        logits = self.classifier(logits).squeeze(-1)

        labels=labels.float()
        loss = None
        if labels is not None:
            # Standard cross-entropy for binary classification (2 classes)
            loss_fct = nn.BCELoss()
            loss = loss_fct(logits, labels)

        return {"loss": loss, "logits": logits}


In [ ]:
classification_model = BartEncoderForClassification(model)
classification_model

BartEncoderForClassification(
  (bart): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-11): 12 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
    

In [ ]:
# Debug check - ensure decoder params are frozen
for name, param in classification_model.named_parameters():
    if "decoder" in name:
        assert param.requires_grad == False


In [ ]:
class StoryClozeDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, texts_1, texts_2, labels):
        self.tokenizer = tokenizer
        self.texts_1 = texts_1
        self.texts_2 = texts_2
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc_1 = self.tokenizer(self.texts_1[idx], padding=True, return_tensors="pt")
        enc_2 = self.tokenizer(self.texts_2[idx], padding=True, return_tensors="pt")

        return {
            "input_ids_1": enc_1["input_ids"].squeeze(0),
            "attention_mask_1": enc_1["attention_mask"].squeeze(0),
            "input_ids_2": enc_2["input_ids"].squeeze(0),
            "attention_mask_2": enc_2["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Example dataset creation
train_dataset = StoryClozeDataset(tokenizer, ClozeTestTraining["Story1"], ClozeTestTraining["Story2"], ClozeTestTraining["AnswerRightEnding"])


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(p):
    preds = p.predictions >= 0.5
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Use the Trainer API

training_args = TrainingArguments(
    report_to="none",
    output_dir="./results",
    per_device_train_batch_size=64,
    num_train_epochs=1,
    save_strategy="no",
)

trainer = Trainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    compute_metrics=compute_metrics

)

trainer.train()

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


Step,Training Loss
500,7.132100
1000,7.112400
1500,7.100100
2000,7.088000
2500,7.076200
3000,7.064700
3500,7.053500
4000,7.042500
4500,7.032000
5000,7.021700


TrainOutput(global_step=29500, training_loss=6.910061465505827, metrics={'train_runtime': 8961.6425, 'train_samples_per_second': 104.389, 'train_steps_per_second': 3.292, 'total_flos': 0.0, 'train_loss': 6.910061465505827, 'epoch': 500.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 6.816177845001221,
 'eval_accuracy': 0.48583645109567075,
 'eval_f1': 0.3177160676589674,
 'eval_runtime': 8.5794,
 'eval_samples_per_second': 218.08,
 'eval_steps_per_second': 27.275,
 'epoch': 500.0}

Now we can get the BART model that is wrapped inside the classification model, for generation and evaluation. Its encoder has been fine tuned with story cloze task. Please note that a lot of evaluation codes is copied from previous fine tuning sections for comparisons. Perhaps it should be encapsulated into a method for elegance. But on the other hand, having them as cells does provide interactivity and some flexibility to enageg with them.

In [ ]:
fine_tuned_model = classification_model.bart
fine_tuned_model.to("cuda")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
%%time
final_perplexity_baseline = 0
final_rouge1_baseline = 0
final_rouge2_baseline = 0
final_rougeL_baseline = 0

inputs_to_be_generated_data = X_test
inputs_to_be_generated_dataloader = DataLoader(inputs_to_be_generated_data, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)
model_to_use = fine_tuned_model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for inputs_to_be_generated, reference in zip(inputs_to_be_generated_dataloader, original_story_endings_reference):
    input_and_attention_mask = tokenizer(inputs_to_be_generated, padding=True, return_tensors="pt")
    input_and_attention_mask.to("cuda")
    generated_ids = model_to_use.generate(inputs=input_and_attention_mask["input_ids"], attention_mask=input_and_attention_mask["attention_mask"], max_length=20, early_stopping=True)
    ending = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    full_story_with_ending = concat_story_body_with_endings(inputs_to_be_generated, ending)
    # Calculate perplexity
    weight_factor = len(reference) / len(inputs_to_be_generated_data)
    final_perplexity_baseline += weight_factor * perplexity.compute(predictions=full_story_with_ending, model_id='facebook/bart-large-cnn')["mean_perplexity"]
    # Calculate rouge
    rouge_results = rouge.compute(predictions=ending, references=reference)
    final_rouge1_baseline += weight_factor * rouge_results["rouge1"]
    final_rouge2_baseline += weight_factor * rouge_results["rouge2"]
    final_rougeL_baseline += weight_factor * rouge_results["rougeL"]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1399: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 3min 27s, sys: 8.67 s, total: 3min 36s
Wall time: 4min 6s


In [ ]:
print(final_perplexity_baseline)
print(final_rouge1_baseline)
print(final_rouge2_baseline)
print(final_rougeL_baseline)

6860325.236742508
0.0
0.0
0.0


In [ ]:
%%time
LLM_as_judge_score_fine_tuned = 0

X_test_inputs_dataloader = DataLoader(TensorDataset(X_test_inputs["input_ids"], X_test_inputs["attention_mask"]), batch_size=32)
original_story_bodies = DataLoader(X_test, batch_size=32)
original_story_endings_reference = DataLoader(y_test, batch_size=32)
model_to_use = fine_tuned_model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

perplexity = evaluate.load("perplexity", module_type="metric")
rouge = evaluate.load('rouge')

with torch.no_grad():
  for input_and_attention_mask, original_stories, reference in zip(X_test_inputs_dataloader, original_story_bodies, original_story_endings_reference):
    generated_ids = model_to_use.generate(inputs=input_and_attention_mask[0], attention_mask=input_and_attention_mask[1], max_length=20, early_stopping=True)
    generated_endings = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    feedback, score = judge.absolute_grade(
      instructions=original_stories,
      responses=generated_endings,
      rubric=score_rubric,
      reference_answers=reference
    )
    LLM_as_judge_score_fine_tuned += sum(score)
LLM_as_judge_score_fine_tuned = LLM_as_judge_score_fine_tuned/len(X_test)
print(LLM_as_judge_score_fine_tuned)

Processed prompts: 100%|██████████| 32/32 [00:16<00:00,  1.94it/s, est. speed input: 1045.70 toks/s, output: 299.10 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:18<00:00,  1.76it/s, est. speed input: 953.93 toks/s, output: 273.73 toks/s]


Processed 32/32 instances.


Processed prompts: 100%|██████████| 32/32 [00:17<00:00,  1.85it/s, est. speed input: 1003.42 toks/s, output: 277.67 toks/s]


...

Processed 32/32 instances.


Processed prompts: 100%|██████████| 15/15 [00:09<00:00,  1.65it/s, est. speed input: 892.20 toks/s, output: 256.45 toks/s]


Processed 15/15 instances.


Finalizing: 100%|██████████| 15/15 [00:00<00:00, 6209.49it/s]

1.0022630834512023
CPU times: user 35min 37s, sys: 5.06 s, total: 35min 43s
Wall time: 35min 31s


In [ ]:
instructions = X_test[0:1]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[0:1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

[' respir deterior surviv massac deletion']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.82s/it, est. speed input: 140.26 toks/s, output: 42.99 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 4132.32it/s]

Feedback: ['The response provided is riddled with incoherent and inappropriate language usage that completely disrupts the narrative flow. The model’s language is not only grammatically incorrect but also includes phrases that are irrelevant to the story, such as "surviv massac deletion." This creates a disjointed and nonsensical ending that does not align with the established narrative about Evan\'s pride in his new BMW. The ending is far from satisfactory, and it does not reflect the fluent and coherent language expected in a suitable ending to a story. Therefore, it fails to meet the criteria outlined in the score rubric, and as a result, it does not elevate the narrative but instead leaves the reader confused and unsatisfied. \n[RESULT] 1']
Score: [1]


In [ ]:
instructions = X_test[1500:1501]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][1500:1501], attention_mask=X_test_inputs["attention_mask"][1500:1501], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[1500:1501]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

[' Recomm deterior funer ticking respir']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.32s/it, est. speed input: 160.45 toks/s, output: 42.45 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5468.45it/s]

Feedback: ["The provided response fails to meet the requirements outlined in the score rubric. The response does not demonstrate language fluency or semantics coherence. It lacks any semblance of a coherent narrative and doesn't follow the story flow. There is a complete disconnect from the provided story body, and it fails to address the main concern of the individual - the pain of the shot. Furthermore, it does not contribute to a satisfactory resolution or enhance the themes and characters of the narrative. The lack of clarity and coherence renders the reader confused and unsatisfied, which is why the response receives a score of 1. \n[RESULT] 1"]
Score: [1]


In [ ]:
instructions = X_test[-2:-1]
responses = tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][-2:-1], attention_mask=X_test_inputs["attention_mask"][-2:-1], max_length=20, early_stopping=True), skip_special_tokens=True)
print(responses)
reference_answers = y_test[-2:-1]

feedback, score = judge.absolute_grade(
    instructions=instructions,
    responses=responses,
    rubric=score_rubric,
    reference_answers=reference_answers
)

print("Feedback:", feedback)
print("Score:", score)

[' Recommttes deterior Tactics massac']


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.45s/it, est. speed input: 157.63 toks/s, output: 42.67 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5809.29it/s]

Feedback: ["The response is lacking in language fluency, with no context provided. It is incoherent and does not follow the story's narrative. The reader is left confused and with no sense of closure or resolution, which significantly disrupts the story's flow. It fails to answer the question of whether Ramona found a new job and instead presents an unrelated phrase with no explanation or relevance to the story. This makes the ending of the narrative unsatisfactory, leaving the reader without the necessary information to understand the outcome of Ramona's job search. As per the score rubric, the response exhibits major language issues and fails to provide a suitable ending to the story. \n[RESULT] 1"]
Score: [1]


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True), skip_special_tokens=True))

[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, do_sample=False), skip_special_tokens=True))

[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']
[' respir deterior surviv massac deletion']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=0.8, do_sample=True), skip_special_tokens=True))

[' deletion survivolate respir LEDs']
[' surviv Recomm deletion respir deterior']
[' respir deterior deletion flattttes']
[' surviv sket deletion respir massac']
[' massac farewell flatt deletion respir']
[' respir farewell deletion funer Tactics']
[' Tactics surviv deterior deletion respir']
[' deletion survivmAh respir deterior']
[' ticking Recomm surviv massacres respir']
[' surviv respir deleg funer massac']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=0.3, do_sample=True), skip_special_tokens=True))

[' surviv Recomm deletion funer respir']
[' respir funer surviv deterior massac']
[' deterior respir surviv deletion massac']
[' respir funer massac surviv deterior']
[' respir surviv funer massac deterior']
[' surviv deterior respir Recomm deletion']
[' Recomm deterior deletion massac respir']
[' deterior respir massacres surviv massac']
[' respir deterior funer surviv Recomm']
[' respir deletion massac surviv deterior']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, temperature=1.5, do_sample=True), skip_special_tokens=True))

[' deletion sket ticking flatt massac']
[' delet surviv LEDs massac thresholds']
[' funer respir surviv massacres subsystem']
[' massac sket massacres silhou Recomm']
[' farewell surviv Tactics funer massac']
[' deterior deletionolate respir massacres']
[' Recomm deleg Tactics deterior ticking']
['mAh deterior redund Tactics deletion']
[' sket respir surviv funer massacres']
[' respir Recomm Tactics massac funer']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=5, do_sample=True), skip_special_tokens=True))

[' respir deterior massac surviv deletion']
[' respir deterior massac surviv deletion']
[' respir deterior surviv funer massac']
[' respir deterior Recomm massac surviv']
[' respir deterior massac funer surviv']
[' deterior respir surviv deletion massac']
[' respir massac deterior surviv funer']
[' respir massac deterior surviv deletion']
[' respir surviv deterior massac deletion']
[' deterior respir funer massac surviv']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=20, do_sample=True), skip_special_tokens=True))

[' deterior deleg respir Tactics massac']
[' respir deterior massac massacres surviv']
[' surviv LEDs respir Recomm farewell']
[' respir deletion surviv funer Recomm']
[' surviv deterior Recomm respir massac']
[' deterior respir LEDs Recomm surviv']
[' deterior respir Tactics buds surviv']
[' Recomm deterior deletion respir LEDs']
[' massac deletion LEDs deterior Recomm']
[' deterior deletion respir surviv funer']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_k=80, do_sample=True), skip_special_tokens=True))

[' respir deletionolateholders funer']
[' deletion LEDs funer deleg deterior']
[' Recomm deterior deleg loudspe funer']
[' flatt ticking massac respir funer']
[' LEDs Tactics deteriorated subsystem surviv']
[' sket Recomm funer farewell deterior']
[' Recomm respir sket Mour surviv']
[' deletion deliveries respirolate surviv']
[' deletion Tactics commem funer Recomm']
[' deletion deterior sket delet massac']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.8, do_sample=True), skip_special_tokens=True))

[' Tactics Recomm surviv respir funer']
[' massac surviv Tactics deterior deletion']
[' funer deletionttes ticking surviv']
[' flatt ticking massac deterior deletion']
[' respir surviv Recomm massacres massac']
[' Tactics Recomm ticking funer surviv']
[' Recomm deterior respir massacres massac']
[' sket respir deterior surviv LEDs']
[' funer respir deleg surviv massacres']
[' massac surviv Tactics deletion Recomm']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.5, do_sample=True), skip_special_tokens=True))

[' respir funer Tactics deletion massac']
[' funer respir deterior massacres Recomm']
[' deterior surviv respir LEDs funer']
[' respir deterior surviv flatt massac']
[' surviv respir Tactics deterior deletion']
[' surviv respir funer Tactics deletion']
[' massac deletion Tactics deterior surviv']
[' respir Tactics flatt deterior deletion']
[' deletion surviv respir massac deterior']
[' surviv deterior respir delet funer']


In [ ]:
for _ in range(10):
  print(tokenizer.batch_decode(fine_tuned_model.generate(X_test_inputs["input_ids"][0:1], attention_mask=X_test_inputs["attention_mask"][0:1], max_length=20, early_stopping=True, top_p=0.3, do_sample=True), skip_special_tokens=True))

[' respir deterior LEDs surviv Recomm']
[' respir deterior surviv massac deletion']
[' respir massac Recomm deterior deletion']
[' respir deterior Recomm surviv massac']
[' respir deterior surviv funer massac']
[' surviv respir massac deterior funer']
[' deterior respir surviv massac deletion']
[' respir LEDs deletion deterior surviv']
[' respir surviv deterior flatt deletion']
[' respir deterior surviv Recomm massac']
